In [6]:
pip install autogluon

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [3]:
pip install fastparquet

Defaulting to user installation because normal site-packages is not writeable
  Obtaining dependency information for fastparquet from https://files.pythonhosted.org/packages/08/7e/94e53ae1d276f24de2fd133dbb2f95cbdf67922b9a3400bba634571182b6/fastparquet-2024.2.0-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata
  Obtaining dependency information for cramjam>=2.3 from https://files.pythonhosted.org/packages/f3/fd/9ad0b88184de652f2f3b6d7abed4f7aebd1154a05cefa1646754aeab4639/cramjam-2.8.2-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 12.5 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 36.3 MB/s eta 0:00:0000:01
Note: you may need to restart the kernel to use updated packages.


In [1]:
import pandas as pd
from pathlib import Path
from sklearn.model_selection import train_test_split

from autogluon.tabular import TabularDataset, TabularPredictor
from autogluon.features.generators import AutoMLPipelineFeatureGenerator, CategoryFeatureGenerator, PipelineFeatureGenerator

from fastparquet import ParquetFile
from utils import feature_drop

In [2]:
file_path = "Data/train_ai_comp_final_dp.parquet"
pf = ParquetFile(file_path)
df = pf.to_pandas()
df = feature_drop(df)
df.head()

,id,target,feature1,feature2,feature3,feature4,feature5,feature6,feature7,feature8,...,feature1067,feature1068,feature1069,feature1070,feature1071,feature1072,feature1073,feature1074,feature1075,feature1076
0,0,0,1761,1759,85,105469,191,46,0,6,...,28913,48985,84264,12045,12107,12510,21126,28913,48985,84264
1,1,0,1761,1759,120,105610,144,71,0,135,...,0,0,0,0,0,0,0,0,0,0
2,2,0,890,1759,141,105227,191,11,0,0,...,55949,70006,113317,33735,31242,12691,48682,55949,70006,113317
5,5,0,1599,966,30,102441,191,8,0,0,...,0,6872,8530,0,0,0,0,0,6872,8530
6,6,0,1761,1759,85,104006,191,2,0,0,...,0,0,7642,0,0,0,0,0,0,7642


In [3]:
X = df.drop(columns=['target', "id"])
y = df['target']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=42)
# X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42)
to_train = pd.concat([X_train, y_train], axis=1)

KeyboardInterrupt: 

In [ ]:
auto_ml_pipeline_feature_generator = AutoMLPipelineFeatureGenerator()
auto_ml_pipeline_feature_generator.fit_transform(X=X_train)

,feature1,feature2,feature3,feature4,feature5,feature6,feature7,feature8,feature9,feature10,...,feature1057,feature1059,feature1060,feature1063,feature1064,feature1065,feature1066,feature1067,feature1068,feature1069
187083,1761,1759,117,105345,191,152,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
585908,1761,1759,88,74109,191,6,125,290,298,176,...,0,0,0,0,0,0,0,0,0,0
241239,1630,601,105,104480,191,34,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
175984,1639,17,86,102228,0,12,125,290,298,176,...,0,0,0,0,0,0,0,0,0,0
393146,1761,1759,32,82893,191,44,125,290,298,176,...,0,20399,21013,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
346164,1633,1759,137,104483,18,38,0,0,211,0,...,0,0,0,2783,0,0,3107,3446,4370,5471
488156,1761,1759,104,104948,78,14,0,92,191,0,...,0,0,0,0,0,0,0,0,18153,47941
176273,1633,17,141,104320,191,67,0,177,57,0,...,0,0,0,0,0,0,0,0,17306,29677
196223,1761,1759,73,97814,93,3,0,0,79,0,...,0,0,0,0,0,0,0,0,0,0


In [ ]:
NUM_HOURS_TO_TRAIN = 2


train_data = TabularDataset(to_train)
predictor = TabularPredictor(label="target", eval_metric="roc_auc").fit(train_data, 
                                                                        time_limit=NUM_HOURS_TO_TRAIN*60*60, 
                                                                        presets="best_quality", 
                                                                        feature_generator=auto_ml_pipeline_feature_generator,
                                                                        num_gpus=3
                                                                        )

# , hyperparameters={'CAT': {}, 'GBM': {}, 'NN': {}}        
        

No path specified. Models will be saved in: "AutogluonModels/ag-20240313_201752"
Presets specified: ['best_quality']
Stack configuration (auto_stack=True): num_stack_levels=1, num_bag_folds=8, num_bag_sets=1
Dynamic stacking is enabled (dynamic_stacking=True). AutoGluon will try to determine whether the input data is affected by stacked overfitting and enable or disable stacking as a consequence.
Detecting stacked overfitting by sub-fitting AutoGluon on the input data. That is, copies of AutoGluon will be sub-fit on subset(s) of the data. Then, the holdout validation data is used to detect stacked overfitting.
Sub-fit(s) time limit is: 14400 seconds.
Starting holdout-based sub-fit for dynamic stacking. Context path is: AutogluonModels/ag-20240313_201752/ds_sub_fit/sub_fit_ho.
Beginning AutoGluon training ... Time limit = 3600s
AutoGluon will save models to "AutogluonModels/ag-20240313_201752/ds_sub_fit/sub_fit_ho"
=================== System Info ===================
AutoGluon Version:  

In [ ]:
to_test = pd.concat([X_test, y_test], axis=1)
test_data = TabularDataset(to_test)
predictor.leaderboard(test_data)

,model,score_test,score_val,eval_metric,pred_time_test,pred_time_val,fit_time,pred_time_test_marginal,pred_time_val_marginal,fit_time_marginal,stack_level,can_infer,fit_order
0,WeightedEnsemble_L2,0.754867,0.758393,roc_auc,9.009816,28.294561,1465.283552,0.008026,0.080873,35.988986,2,True,5
1,LightGBMXT_BAG_L1,0.754610,0.755628,roc_auc,4.652645,15.746282,886.132034,4.652645,15.746282,886.132034,1,True,3
2,LightGBM_BAG_L1,0.753097,0.756096,roc_auc,4.349145,12.467406,543.162532,4.349145,12.467406,543.162532,1,True,4
3,KNeighborsDist_BAG_L1,0.546609,0.554933,roc_auc,38.997111,752.858770,79.094816,38.997111,752.858770,79.094816,1,True,2
4,KNeighborsUnif_BAG_L1,0.546186,0.554798,roc_auc,42.165140,1299.299659,66.448220,42.165140,1299.299659,66.448220,1,True,1


In [10]:
predictor.feature_importance(test_data)

These features in provided data are not utilized by the predictor and will be ignored: ['feature13', 'feature20', 'feature28', 'feature211', 'feature213', 'feature221', 'feature223', 'feature228', 'feature236', 'feature245', 'feature281', 'feature314', 'feature321', 'feature323', 'feature365', 'feature388', 'feature389', 'feature419', 'feature430', 'feature431', 'feature466', 'feature573', 'feature580', 'feature581', 'feature582', 'feature583', 'feature585', 'feature586', 'feature593', 'feature618', 'feature619', 'feature645', 'feature662', 'feature679', 'feature682', 'feature700', 'feature711', 'feature719', 'feature729', 'feature744', 'feature774', 'feature789', 'feature798', 'feature823', 'feature835', 'feature889', 'feature910', 'feature1040', 'feature1041', 'feature1044', 'feature1047', 'feature1048', 'feature1058', 'feature1061', 'feature1062', 'feature1070', 'feature1071', 'feature1072', 'feature1073', 'feature1074', 'feature1075', 'feature1076']
Computing feature importance via

KeyboardInterrupt: 

In [ ]:
predictor.save('dir_save')